# DreamBooth RunPod Notebook

Run Notebook on a machine with sufficient GPU VRAM.

### To run this notebook in RunPod:
```
!git clone https://github.com/NikolaAtanasov/Dreambooth-Stable-Diffusion
```

Forked from:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion/blob/main/dreambooth_runpod_joepenna.ipynb



# 1.  Build Environment
Most packages should already be installed if instancce is a SD template

In [ ]:
%cd Dreambooth-Stable-Diffusion

!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1
# additional


# 2. Download model (a or b)

## 2.a Download model from GDrive
Right-click on the .ckpt file in GDrive, "Get link", set the link access for anyone with the link, copy the id of the file from the url (between /d/{file id}/view) and paste it in the cell below

In [ ]:
!gdown https://drive.google.com/uc?id=

## 2.b Download model from Hugging Face (TODO)

In [ ]:
# set API token

In [ ]:
# download desired model - provide list

# 3. Regularization images (a or b)

### 3.a Either download pre-generated regularization images
`man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5   
`man_unsplash` - pictures from various photographers   
`person_ddim` `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0   
`person_ddim` is recommended

In [ ]:
#Download Regularization Images

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "blonde_woman"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

### 3.b Or generate class images
Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [ ]:
# GENERATE 200 images - Optional
self_generated_files_prompt = "person" #@param {type:"string"}
self_generated_files_count = 200 #@param {type:"integer"}

!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter {self_generated_files_count} \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt {self_generated_files_prompt}

dataset=self_generated_files_prompt

!mkdir -p regularization_images/{dataset}
!mv outputs/txt2img-samples/*.png regularization_images/{dataset}

# 4. Upload training images
Upload 10-20 images of the target to:  /workspace/Dreambooth-Stable-Diffusion/training_images   
WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.
*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be as close as possible to the kind of images you're trying to make

In [ ]:
mkdir ./training_images

Drag and drop selected images to the new training_images folder.   
(Should be an even number of 512x512 PNG images)

# 5. Training


In [ ]:
#@markdown This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name" #@param {type:"string"}

# MAX STEPS
#@markdown ~100-200 x N (N = number of training images provided)
max_training_steps = 2000 #@param {type:"integer"}

#@markdown Match class_word to the category of the regularization images you chose above.
class_word = "person" #@param ["man", "person", "woman"] {allow-input: true}

#@markdown This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName" #@param {type:"string"}
reg_data_root = "/content/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/content/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

### Troubleshooting

CUDA out of memory: open a new terminal, `ps aux` to check running processes, `kill` "launcher" and "webui" related ones (usually 11, 25, 26)

# 6. Copy and name the checkpoint file

In [ ]:
directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

# 7. Upload model to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp trained_models/{file_name} /content/drive/MyDrive/{file_name}